In [1]:
from sklearn.datasets import load_boston
import pandas as pd
import numpy as np
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.svm import SVC, LinearSVC
from sklearn.linear_model import LinearRegression,Ridge, Lasso, LogisticRegression
from sklearn.model_selection import GridSearchCV, cross_val_score

import warnings
warnings.filterwarnings("ignore")

In [2]:
RANDOM_STATE = 42

In [3]:
dataset = load_boston()
X = pd.DataFrame(dataset.data)
X.columns = dataset.feature_names
y = dataset.target

1. Разделите выборку на обучающую и тестовую в отношении 80%/20%

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    train_size=0.2, 
                                                    random_state=42)

2. Обучите стандартную регрессию, а также Ridge и  Lasso и параметрами по умолчанию и выведите их R2 на тестовой выборке

In [5]:
from sklearn.linear_model import LinearRegression,Ridge, Lasso
from sklearn.metrics import mean_squared_error, r2_score
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)
y_pred_lr = lr_model.predict(X_test)

r2 = r2_score(y_test, y_pred_lr)
print(f"R2: {r2:.2f}")

ridge_model = Ridge()
ridge_model.fit(X_train, y_train)
y_pred_ridge = ridge_model.predict(X_test)

r2 = r2_score(y_test, y_pred_ridge)
print(f"R2: {r2:.2f}")

lasso_model = Lasso()
lasso_model.fit(X_train, y_train)
y_pred_lasso = lasso_model.predict(X_test)

r2 = r2_score(y_test, y_pred_lasso)
print(f"R2: {r2:.2f}")

R2: 0.71
R2: 0.71
R2: 0.65


 3. Для Ridge и Lasso подберите коэффициент регуляризации(используйте GridSearchCV, RidgeCV, LassoCV) в пределах от $10^{-5}$ до $10^5$ (по степеням 10). Посчиrтайте R2 на тестовой выборке по лучшим моделям и сравните с предыдущими результатами. Напишите как изменился результат

In [6]:
from sklearn.linear_model import RidgeCV, LassoCV
from sklearn.model_selection import GridSearchCV, cross_val_score
model = Ridge()

parameters = {"alpha":[1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1e+1, 1e+2, 1e+3, 1e+4, 1e+5]}
Ridge_reg= GridSearchCV(model, parameters, scoring='r2',cv=5)

result = Ridge_reg.fit(X_train, y_train)
print(result.best_params_)
print(result.best_score_)



{'alpha': 10.0}
0.5341051092937275


In [7]:
model = Lasso()

parameters = {"alpha":[1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1e+1, 1e+2, 1e+3, 1e+4, 1e+5]}
Lasso_reg= GridSearchCV(model, parameters, scoring='r2',cv=5)

result = Lasso_reg.fit(X_train, y_train)
print(result.best_params_)
print(result.best_score_)

{'alpha': 0.1}
0.5112232840698209


In [8]:
reg = RidgeCV([1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1e+1, 1e+2, 1e+3, 1e+4, 1e+5], scoring='r2', cv=5)
reg.fit(X_train, y_train)

reg.fit(X_train, y_train)
reg.score(X_test, y_test)

0.7065995399152001

In [9]:
model = LassoCV(alphas=[1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1e+1, 1e+2, 1e+3, 1e+4, 1e+5], cv=5)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
r2 = r2_score(y_test, y_pred)
r2

0.7049260440433984

4. Проведите масштабирование выборки(используйте Pipeline, StandardScaler, MinMaxScaler), посчитайте R2 и сравните с предыдущими результатами. Напишите как изменился результат

In [10]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler

pipe = make_pipeline(MinMaxScaler(), Ridge())
pipe.fit(X_train, y_train)

y_pred = pipe.predict(X_test)
r2 = r2_score(y_test, y_pred)
print(f'R2_ridge_minmax {r2:.2f}')

pipe = make_pipeline(StandardScaler(), Ridge())
pipe.fit(X_train, y_train)

y_pred = pipe.predict(X_test)
r2 = r2_score(y_test, y_pred)
print(f'R2_ridge_st {r2:.2f}')

R2_ridge_minmax 0.70
R2_ridge_st 0.71


In [11]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler

pipe = make_pipeline(MinMaxScaler(), Lasso())
pipe.fit(X_train, y_train)

y_pred = pipe.predict(X_test)
r2 = r2_score(y_test, y_pred)
print(f'R2_lasso_minmax {r2:.2f}')

pipe = make_pipeline(StandardScaler(), Lasso())
pipe.fit(X_train, y_train)

y_pred = pipe.predict(X_test)
r2 = r2_score(y_test, y_pred)
print(f'R2_lasso_st {r2:.2f}')

R2_lasso_minmax 0.21
R2_lasso_st 0.66


5. Подберите коэффициент регуляризации для Ridge и Lasso на масштабированных данных, посчитайте R2 и сравните с предыдущими результатами. Напишите как изменился результат

In [12]:
reg = RidgeCV([1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1e+1, 1e+2, 1e+3, 1e+4, 1e+5], scoring='r2', cv=5)
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler

pipe = make_pipeline(MinMaxScaler(), reg)
pipe.fit(X_train, y_train)

y_pred = pipe.predict(X_test)
r2 = r2_score(y_test, y_pred)
print(f'R2_ridge_minmax {r2:.2f}')

pipe = make_pipeline(StandardScaler(), reg)
pipe.fit(X_train, y_train)

y_pred = pipe.predict(X_test)
r2 = r2_score(y_test, y_pred)
print(f'R2_ridge_st {r2:.2f}')

R2_ridge_minmax 0.71
R2_ridge_st 0.71


In [13]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
reg = LassoCV(alphas=[1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1e+1, 1e+2, 1e+3, 1e+4, 1e+5], cv=5)

pipe = make_pipeline(MinMaxScaler(), reg)
pipe.fit(X_train, y_train)

y_pred = pipe.predict(X_test)
r2 = r2_score(y_test, y_pred)
print(f'R2_lasso_minmax {r2:.2f}')

pipe = make_pipeline(StandardScaler(), reg)
pipe.fit(X_train, y_train)

y_pred = pipe.predict(X_test)
r2 = r2_score(y_test, y_pred)
print(f'R2_lasso_st {r2:.2f}')

R2_lasso_minmax 0.68
R2_lasso_st 0.71


6. Добавьте попарные произведения признаков и их квадраты (используйте PolynomialFeatures) на масштабированных признаках, посчитайте R2 и сравните с предыдущими результатами. Напишите как изменился результат

In [14]:
reg = RidgeCV([1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1e+1, 1e+2, 1e+3, 1e+4, 1e+5], scoring='r2', cv=5)
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(2)

pipe = make_pipeline(MinMaxScaler(), poly, reg)
pipe.fit(X_train, y_train)

y_pred = pipe.predict(X_test)
r2 = r2_score(y_test, y_pred)
print(f'R2_ridge_minmax {r2:.2f}')

pipe = make_pipeline(StandardScaler(), poly, reg)
pipe.fit(X_train, y_train)

y_pred = pipe.predict(X_test)
r2 = r2_score(y_test, y_pred)
print(f'R2_ridge_st {r2:.2f}')

R2_ridge_minmax 0.78
R2_ridge_st 0.53


In [15]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
reg = LassoCV(alphas=[1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1e+1, 1e+2, 1e+3, 1e+4, 1e+5], cv=5)
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(2)

pipe = make_pipeline(MinMaxScaler(), poly, reg)
pipe.fit(X_train, y_train)

y_pred = pipe.predict(X_test)
r2 = r2_score(y_test, y_pred)
print(f'R2_lasso_minmax {r2:.2f}')

pipe = make_pipeline(StandardScaler(), poly, reg)
pipe.fit(X_train, y_train)

y_pred = pipe.predict(X_test)
r2 = r2_score(y_test, y_pred)
print(f'R2_lasso_st {r2:.2f}')

R2_lasso_minmax 0.73
R2_lasso_st 0.75


7. Подберите наилучшую модель (используйте Pipeline, GridSearchSCV) подбирая тип регуляризации (L1,L2), коэффициент регуляризации, метод масштабирования и степень полинома в PolynomialFeatures. Выведите итоговые параметры и результат R2. Напишите как изменился R2 по сравнению с предыдущими экспериментами

In [16]:
model = Ridge()

poly = PolynomialFeatures(2)
parameters = {"alpha":[1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1e+1, 1e+2, 1e+3, 1e+4, 1e+5]}
reg= GridSearchCV(model, parameters, scoring='r2',cv=5)
pipe = make_pipeline(MinMaxScaler(), poly, reg)
pipe.fit(X_train, y_train)
print(pipe[2].best_params_)

y_pred = pipe.predict(X_test)
r2 = r2_score(y_test, y_pred)
print(f'R2_minmax {r2:.2f}')

pipe = make_pipeline(StandardScaler(), poly, reg)
pipe.fit(X_train, y_train)
print(pipe[2].best_params_)


y_pred = pipe.predict(X_test)
r2 = r2_score(y_test, y_pred)
print(f'R2_st {r2:.2f}')

{'alpha': 0.1}
R2_minmax 0.78
{'alpha': 10.0}
R2_st 0.53


http://archive.ics.uci.edu/ml/datasets/Adult

In [17]:
link = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/adult-all.csv'
data = pd.read_csv(link, header=None)

In [18]:
data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


8. Разделите выборку на признаки и целевую переменную(колонка со зачениями {<=50K,>50K}). Замените целевую переменную на числовые значения.

In [19]:
data[14].loc[data[14] == '<=50K'] = 0
data[14].loc[data[14] == '>50K'] = 1

In [20]:
y = data[14]
y= y.astype('int')
X = data.drop(columns=[14])


9. Выясните, присутствуют ли в данных пропуски. Заполните их самыми частыми значениями (испольуйте SimpleImputer)

In [24]:
from sklearn.impute import SimpleImputer
#Пропусков нет, заменять вопросы буду в 14 задании
#X_nan = X.replace([0,'?'], np.NaN)
print(X.isnull().sum())
#imp = SimpleImputer(strategy='most_frequent')
#X_fill = imp.fit_transform(X_nan)


0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
dtype: int64


10. Выберите колонки с числовыми и категориальными переменными.

In [25]:
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object']).columns

11. Создайте пайплайн по обработке колонок(используйте OneHotEncoder,MinMaxScaler).

In [26]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn import model_selection

numeric_transformer = Pipeline(steps=[
    #('imputer', SimpleImputer(strategy='most_frequent')),
    ('scaler', MinMaxScaler())])

categorical_transformer = Pipeline(steps=[
    #('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder())])

# Use the ColumnTransformer to apply to the correct features
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
        ])

result = preprocessor.fit_transform(X)

12. Посчитайте метрики accuracy и f1_score на предсказании только самого частого класса в целевой переменной.

In [27]:
X_train, X_test, y_train, y_test = train_test_split(
    result, y, test_size=0.2, random_state=RANDOM_STATE)
LR = LinearRegression()
LR.fit(X_train, y_train)
y_pred = LR.predict(X_test).round()

In [28]:
acc = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')

In [29]:
print(acc, f1)

0.8338622172177296 0.8210298594639663


13. Посчитайте cross_val_score по алгоритмам LogisticRegression, SVC, LinearSVC по метрикам accuracy и f1_score.
Напишите удалось ли превзойти предыдущий результат.

In [31]:
reg = LogisticRegression(max_iter=100).fit(result,y)
acc = cross_val_score(reg, result, y, scoring='accuracy', cv=5)

In [32]:
f1 = cross_val_score(reg, result, y, scoring='f1', cv=5)

In [33]:
print(acc.max(), f1.max())

0.8568796068796068 0.6665076335877863


In [34]:
reg = SVC().fit(result,y)
acc = cross_val_score(reg, result, y, scoring='accuracy', cv=5)
f1 = cross_val_score(reg, result, y, scoring='f1', cv=5)

In [35]:
print(acc.max(), f1.max())

0.845004095004095 0.6318840579710144


In [36]:
reg = LinearSVC().fit(result,y)
acc = cross_val_score(reg, result, y, scoring='accuracy', cv=5)
f1 = cross_val_score(reg, result, y, scoring='f1', cv=5)
print(acc.max(), f1.max())

0.8572891072891073 0.6645813282001924


14. Можно заметить что в данных присутствуют значения '?', замените их самыми частыми значениями (испольуйте SimpleImputer)

In [37]:
numeric_transformer = Pipeline(steps=[
    #('imputer', SimpleImputer(strategy='most_frequent')),
    ('scaler', MinMaxScaler())])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(missing_values='?',strategy='most_frequent')),
    ('onehot', OneHotEncoder())])

# Use the ColumnTransformer to apply to the correct features
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
        ])

result_1 = preprocessor.fit_transform(X)

15. Посчитайте cross_val_score на новых данных. Напишите удалось ли улучшить результат.

In [40]:
reg = LogisticRegression(max_iter=100).fit(result_1,y)
acc = cross_val_score(reg, result_1, y, scoring='accuracy', cv=5)
f1 = cross_val_score(reg, result_1, y, scoring='f1', cv=5)
print(acc.max(), f1.max())
#даже на 0.001 ухудшилось

0.8557534807534808 0.664922711058264


In [41]:
reg = SVC().fit(result_1,y)
acc = cross_val_score(reg, result_1, y, scoring='accuracy', cv=5)
f1 = cross_val_score(reg, result_1, y, scoring='f1', cv=5)
print(acc.max(), f1.max())
#уменшились значения

0.8428542178542179 0.6266407389402042


In [42]:
reg = LinearSVC().fit(result_1,y)
acc = cross_val_score(reg, result_1, y, scoring='accuracy', cv=5)
f1 = cross_val_score(reg, result_1, y, scoring='f1', cv=5)
print(acc.max(), f1.max())
#уменшились значения

0.8542178542178542 0.6570327552986512


16. Посчитайте cross_val_score, если просто удалить значения '?'. Напишите как изменился результат

In [43]:
X_nan = X.replace(['?'], np.NaN)
numeric_transformer = Pipeline(steps=[
    #('imputer', SimpleImputer(strategy='most_frequent')),
    ('scaler', MinMaxScaler())])

categorical_transformer = Pipeline(steps=[
    #('imputer', SimpleImputer(missing_values='?',strategy='most_frequent')),
    ('onehot', OneHotEncoder())])

# Use the ColumnTransformer to apply to the correct features
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
        ])

result_1 = preprocessor.fit_transform(X_nan)

reg = LogisticRegression(max_iter=100).fit(result_1,y)
acc = cross_val_score(reg, result_1, y, scoring='accuracy', cv=5)
f1 = cross_val_score(reg, result_1, y, scoring='f1', cv=5)
print(acc.max(), f1.max())

reg = SVC().fit(result_1,y)
acc = cross_val_score(reg, result_1, y, scoring='accuracy', cv=5)
f1 = cross_val_score(reg, result_1, y, scoring='f1', cv=5)
print(acc.max(), f1.max())

reg = LinearSVC().fit(result_1,y)
acc = cross_val_score(reg, result_1, y, scoring='accuracy', cv=5)
f1 = cross_val_score(reg, result_1, y, scoring='f1', cv=5)
print(acc.max(), f1.max())

#значения улучшились, если их просто удалить

0.8569819819819819 0.6668256618173145
0.845004095004095 0.6318840579710144
0.8572891072891073 0.6645813282001924


 17. Посчитайте cross_val_score для RandomForestClassifier,GradientBoostingClassifier. Напишите как изменился результат и какой вывод можно из этого сделать.

In [46]:
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
numeric_transformer = Pipeline(steps=[
    #('imputer', SimpleImputer(strategy='most_frequent')),
    ('scaler', MinMaxScaler())])

categorical_transformer = Pipeline(steps=[
    #('imputer', SimpleImputer(missing_values='?',strategy='most_frequent')),
    ('onehot', OneHotEncoder())])

# Use the ColumnTransformer to apply to the correct features
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
        ])

result_1 = preprocessor.fit_transform(X_nan)

reg = RandomForestClassifier().fit(result_1,y)
acc = cross_val_score(reg, result_1, y, scoring='accuracy', cv=5)
f1 = cross_val_score(reg, result_1, y, scoring='f1', cv=5)
print(acc.max(), f1.max())

reg = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=RANDOM_STATE).fit(result_1,y)
acc = cross_val_score(reg, result_1, y, scoring='accuracy', cv=5)
f1 = cross_val_score(reg, result_1, y, scoring='f1', cv=5)
print(acc.max(), f1.max())

0.8558558558558559 0.6733178654292343
0.8658886158886159 0.6904315196998124


18. Подберите наилучшую модель, подбирая методы обработки колонок - масштабирование признаков, кодирование признаков и заполнение пропусков. Параметры алгоритмов оставьте по умолчанию. Выведите итоговые параметры и результат accuracy и f1_score.

In [49]:
print(acc.max(), f1.max())
reg.get_params()

0.8658886158886159 0.6904315196998124


{'ccp_alpha': 0.0,
 'criterion': 'friedman_mse',
 'init': None,
 'learning_rate': 1.0,
 'loss': 'deviance',
 'max_depth': 1,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_iter_no_change': None,
 'random_state': 42,
 'subsample': 1.0,
 'tol': 0.0001,
 'validation_fraction': 0.1,
 'verbose': 0,
 'warm_start': False}